In [4]:
import random

class Account(object):
    '''
    Base Account Object for all types of accounts.
    '''
    _overdraft_fee = 35.0

    def __init__(self, balance, account_type, _account_no=None):
        if _account_no is None:
            self._account_no = self.gen_account_num()
        else:
            self._account_no = _account_no
            
        self.balance = float(balance)
        self.account_type = account_type
        
    def __str__(self):
        return '{0.account_type} #{0._account_no!s} - ${0.balance!s}'.format(self)
    
    def __repr__(self):
        return '{self.__class__.__name__}({self.balance}, {self.account_type})'.format(self=self)
    
    @staticmethod
    def gen_account_num():
        gen_number = random.randint(100000000000000, 9999999999999999)
        return gen_number
    
    @property
    def get_funds(self):
        return self.balance
    
    def check_withdraw(self, amount):
        proposal = (self.balance - amount)
        return True if proposal >= float(0) else False
    
    def withdraw(self, amount):
        if self.check_withdraw(amount) is True:
            self.balance -= amount
            return self.balance
        else:
            raise ValueError("Not enough funds.")
    
    def deposit(self, amount):
        self.balance += amount
        return self.balance
    
    def calc_interest(self):
        interest = self.balance * 0.01 
        total = self.balance * 1.01
        self.balance = total
        return self.balance
    
    def get_standing(self):
        return True if self.balance >= 1000 else False
    
    def make_record(self, filename='account.log'):
        with open(filename, 'a') as file:
            file.write('{0._account_no},{0.balance},{0.account_type}'.format(self))
            
    @classmethod
    def from_csv_string(Klass, record):
        record = record.split(',')
        account_no, balance, account_type = record[0], record[1], record[2]
        return Klass(balance, account_type, _account_no=account_no)

In [5]:
import unittest


class TestAccountCreation(unittest.TestCase):
    
    def test_initial_account_open(self):
        account = Account(210, 'checking')
        self.assertEqual(account.balance, 200)
        self.assertEqual(account.account_type, 'checking')        

In [6]:
def read_account_file(filename='./account.log'):
    '''
    Reads the account objects file and returns a list of account csv strings.
    '''
    with open(filename, 'r') as file:
        records = file.readlines()
        return records
    
def load_accounts():
    '''
    Loads the account file objects, and returns a dictionary keyed by account number.
    '''
    account_lines = read_account_file()
    accounts_dict = dict()
    
    for line in account_lines:
        account = Account.from_csv_string(line)
        accounts_dict[account._account_no] = account
        
    return accounts_dict


def select_account(accounts, last_four):
    '''
    Returns an account object loaded from an account file matching the last four digits.
    '''
    for account in accounts:
        if accounts[account]._account_no[-4::1] == last_four:
            selected_account = accounts[account]
            print('Selected account #{}'.format(account))
            break
    else:
        print('No account found with last four digits {0!s}'.format(account_select))
        return None
    
    return selected_account


def atm(accounts):
    '''
    The ATM keypad interface logic.
    '''
    print(list(accounts))
    last_four = input('Enter last four digits of account no. ')
    account = select_account(accounts, last_four)
    
    print(account, account._account_no, account.balance, account.get_funds(), end='\n')
    print('1 - Check Balance',
          '2 - Deposit',
          '3 - Withdraw',
          sep='\n',
          end='\n')
    
    keypad = int(input('Enter 1-3'))
    
    if keypad == 1: 
        print('Getting Balance...')
        print('${0!s}'.format(account.balance))
        
    elif keypad == 2:
        deposit_amount = float(input('Enter deposit amount'))
        account.deposit(deposit_amount)
        print('${0!s}'.format(account.balance))
        
    elif keypad == 3:
        withdraw_amount = float(input('Enter amount to withdraw'))
        account.withdraw(withdraw_amount)
        print('${0!s}'.format(account.balance))
        
    else:
        print("Invaid response. Try again")
        
    return accounts
        
        
def main():
    '''
    Runs the Atm interface in a loop, until the file exits.
    '''
    print('Welcome.', end='\n')
    # Loads accounts from the accounts file.
    accounts = load_accounts()
    
    more = 'Y'
    while more.upper() == 'Y':
        accounts = atm(accounts)
        print(accounts)
        more = input("Perform Another Transaction? y/n")
        if more != 'y' and more != 'n':
            print("Enter y or n.")
            continue
        
    else:
        print('Goodbye!')
        for account in accounts:
            accounts[account].make_record()
    

In [7]:
main()

Welcome.
['222623244180333']
Enter last four digits of account no. 0333
Selected account #222623244180333
checking
 #222623244180333 - $100011333.13 222623244180333 100011333.13 100011333.13
1 - Check Balance
2 - Deposit
3 - Withdraw
Enter 1-31
Getting Balance...
$100011333.13
{'222623244180333': Account(100011333.13, checking
)}
Perform Another Transaction? y/ny
['222623244180333']
Enter last four digits of account no. 0333
Selected account #222623244180333
checking
 #222623244180333 - $100011333.13 222623244180333 100011333.13 100011333.13
1 - Check Balance
2 - Deposit
3 - Withdraw
Enter 1-33
Enter amount to withdraw9999999
$90011334.13
{'222623244180333': Account(90011334.13, checking
)}
Perform Another Transaction? y/nn
Goodbye!
